In [2]:
from bs4 import BeautifulSoup
import re
import csv

In [3]:
html_path = 'LEAD_ISA_2013B.html'
csv_path = 'LEAD_2013_citation_table_raw.csv'

In [4]:
f = open(html_path, 'r', encoding='cp1252')
contents = f.read()
soup = BeautifulSoup(contents,'html.parser')
soup.prettify(formatter=lambda s: s.replace(u'\xa0', u'').replace("&nbsp;", ""))

f.close()

In [5]:
def is_heading(s):
    """"
    if tag.name.startswith('h'):
        return True
    
    if 'class' in tag.attrs:
        if  'Head0' in tag.attrs['class']:
            return True
    """
    if '\u03BC' in s:
        return False
    if s.startswith('Executive') and 'Summary' in s and '________' not in s:
        return True
    if re.search(r'^Preamble\s*$',s) is not None and  '________' not in s:
        return True
    if re.search(r'^[1-6](\.\d+)+\s*[A-Z]\.?\w+', s.replace(u'\xa0', '')) is not None and '________' not in s:
        return True
    if re.search(r'^CHAPTER \d ',s) is not None and '________' not in s :
        return True

In [6]:
def write_out_one_row(i, hero_id, level1, level2, level3, level4, level5, level6, subtag_text, tag_text, csv_r):
    if level1 == 'PREFACE' or level1.startswith('EXECUTIVE'): # chapter, section, and subsections were from the TOC before FREFACE
            s_level2, s_level3, s_level4, s_level5, s_level6 = '-', '-', '-', '-', '-'

    else:
          s_level2 = level2

    if level3.startswith(level2.split()[0]):
        s_level3 = level3
        if level4.startswith(level3.split()[0]):
            s_level4 = level4
            if level5.startswith(level4.split()[0]):
                s_level5 = level5
                if level6.startswith(level5.split()[0]):
                    s_level6 = level6
                else:
                    s_level6 = '-'
            else:
                s_level5 = '-'
                s_level6 = '-'
        else:
            s_level4 = '-'
            s_level5 = '-'
            s_level6 = '-'
    else:
        s_level3 = '-'
        s_level4 = '-'
        s_level5 = '-'
        s_level6 = '-'



      # print("write row")
    row = [str(i), hero_id, level1, s_level2, s_level3, s_level4, s_level5, s_level6, subtag_text, tag_text]
    csv_r.writerow(row)

In [10]:
def process_text(t):
    t = t.replace('‘', "'").replace('’', "'").replace('“', '"').replace('”', '"').replace('‑', '-').replace('·', '').replace('־', '-').replace('−', '-').replace('×', 'x')
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [11]:
f = open(csv_path, 'w', encoding='utf-8',newline='')
csv_r = csv.writer(f)
i = 0
level1 = '-'
level2 = '-'
level3 = '-'
level4 = '-'
level5 = '-'
level6 = '-'

row = ['INSTANCE_ID', 'REFERENCE_ID', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5', 'Level6', 'AnchorText', 'ContextParagraph']
csv_r.writerow(row)

#for tag in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8']):
    # print(f'{tag.name}: {tag.text}')

    
    
for tag in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8','table']):
    ishead = False
    tag_text = process_text(tag.text)
    table_text = []
    if tag.name =='table':
        
        trs = tag.find_all(['tr'])
        for tr in trs:
            tbodys = tr.find_all(['p'])
            if len(tbodys)>=2:
                if re.search(r'^[1-6](\.\d+)+\s*$',process_text(tbodys[0].text)):
                    ishead = True
                    num = process_text(tbodys[0].text)                  
                    title = process_text(tbodys[1].text)
                    tag_text = tag_text[tag_text.index(num):]
                    table_text.append((num, tag_text))
                    #table_text.append(tbodys[0])
    else:
        tag_text = process_text(tag.text)

        if is_heading(tag.text):
            num = tag_text.split()[0]
            ishead = True
            
    #if num=='1.7':
        #print(len(num.split('.')))
    if tag.name =='table' and ishead:
        
        for i in range(len(table_text)):
            num = table_text[i][0]
            tag_text = table_text[i][1]
            if ishead and len(num.split('.')) == 1:
                level1 = tag_text
                # print ('PART', part)

            if ishead and len(num.split('.')) == 2:
                    level2 = tag_text
                    # print ('CHAPTER', chapter)

            if ishead and len(num.split('.')) == 3:
                    level3 = tag_text
                    # print ('SECTION', section)

            if ishead and len(num.split('.')) == 4:
                    level4 = tag_text
                    # print ('SUBSECTION', subsection)

            if ishead and len(num.split('.')) == 5:
                    level5 = tag_text

            if ishead and len(num.split('.')) == 6:
                    level6 = tag_text

            pre_hero_id=""
            pre_tag_text=""
            anchor_text = ""

            for subtag in tag.descendants: # write out all citations in the current paragraph
                
                if subtag.name == 'a' and 'href' in subtag.attrs:
                    if 'hero' in subtag.attrs['href'] and 'reference_id' in subtag.attrs['href']:

                        hero_id = subtag.attrs['href'].split('=')[-1]

                       
                        if re.search(r'\d+', hero_id) is not None:
                            subtag_text = process_text(subtag.text)

                            if pre_hero_id == hero_id and tag_text == pre_tag_text:
                                anchor_text += subtag_text
                                
                            else:
                                anchor_text = subtag_text
                          
                          #   if level1 == 'PREFACE' or level1.startswith('EXECUTIVE'): # chapter, section, and subsections were from the TOC before FREFACE
                          #       s_level2, s_level3, s_level4, s_level5, s_level6 = '-', '-', '-', '-', '-'
                          #   else:
                          #       s_level2 = level2
                       

                          #   if level3.startswith(level2.split()[0]):
                          #       s_level3 = level3
                          #       if level4.startswith(level3.split()[0]):
                          #           s_level4 = level4
                          #           if level5.startswith(level4.split()[0]):
                          #               s_level5 = level5
                          #               if level6.startswith(level5.split()[0]):
                          #                   s_level6 = level6
                          #               else:
                          #                   s_level6 = '-'
                          #           else:
                          #               s_level5 = '-'
                          #               s_level6 = '-'
                          #       else:
                          #           s_level4 = '-'
                          #           s_level5 = '-'
                          #           s_level6 = '-'
                          #   else:
                          #       s_level3 = '-'
                          #       s_level4 = '-'
                          #       s_level5 = '-'
                          #       s_level6 = '-'

                          #   subtag_text = process_text(subtag.text)
                          #  # print("write row")
                          #   row = [str(i), hero_id, level1, s_level2, s_level3, s_level4, s_level5, s_level6, subtag_text, tag_text]
                          #   csv_r.writerow(row)
                            
                            write_out_one_row(i, hero_id, level1, level2, level3, level4, level5, level6, anchor_text, tag_text, csv_r)
                            pre_hero_id = hero_id
                            pre_tag_text = tag_text

                                                                                
    else:
        if ishead and len(num.split('.')) == 1:
                level1 = tag_text
                # print ('PART', part)

        if ishead and len(num.split('.')) == 2:
                level2 = tag_text
                # print ('CHAPTER', chapter)

        if ishead and len(num.split('.')) == 3:
                level3 = tag_text
                # print ('SECTION', section)

        if ishead and len(num.split('.')) == 4:
                level4 = tag_text
                # print ('SUBSECTION', subsection)

        if ishead and len(num.split('.')) == 5:
                level5 = tag_text

        if ishead and len(num.split('.')) == 6:
                level6 = tag_text

        pre_hero_id=""
        pre_tag_text=""


       # print(num,' '," ")
        anchor_text = ""
        for subtag in tag.descendants: # write out all citations in the current paragraph

            
            if subtag.name == 'a' and 'href' in subtag.attrs:
                if 'hero' in subtag.attrs['href'] and 'reference_id' in subtag.attrs['href']:

                    hero_id = subtag.attrs['href'].split('=')[-1]



                    # if hero_id == pre_hero_id and tag_text == pre_tag_text:
                    #     continue
                    # 
                    # pre_tag_text = tag_text
                    # #print(hero_id)
                    if re.search(r'\d+', hero_id) is not None:

                        subtag_text = process_text(subtag.text)

                        if pre_hero_id == hero_id and tag_text == pre_tag_text:
                            anchor_text += subtag_text
                            
                        else:
                            anchor_text = subtag_text

                      #   if level1 == 'PREFACE' or level1.startswith('EXECUTIVE'): # chapter, section, and subsections were from the TOC before FREFACE
                      #       s_level2, s_level3, s_level4, s_level5, s_level6 = '-', '-', '-', '-', '-'
                      #   else:
                      #       s_level2 = level2

                      #   if level3.startswith(level2.split()[0]):
                      #       s_level3 = level3
                      #       if level4.startswith(level3.split()[0]):
                      #           s_level4 = level4
                      #           if level5.startswith(level4.split()[0]):
                      #               s_level5 = level5
                      #               if level6.startswith(level5.split()[0]):
                      #                   s_level6 = level6
                      #               else:
                      #                   s_level6 = '-'
                      #           else:
                      #               s_level5 = '-'
                      #               s_level6 = '-'
                      #       else:
                      #           s_level4 = '-'
                      #           s_level5 = '-'
                      #           s_level6 = '-'
                      #   else:
                      #       s_level3 = '-'
                      #       s_level4 = '-'
                      #       s_level5 = '-'
                      #       s_level6 = '-'

                        

                      #  # print("write row")
                      #   row = [str(i), hero_id, level1, s_level2, s_level3, s_level4, s_level5, s_level6, subtag_text, tag_text]
                      #   csv_r.writerow(row)

                        write_out_one_row(i, hero_id, level1, level2, level3, level4, level5, level6, anchor_text, tag_text, csv_r)
                        
                        pre_hero_id = hero_id
                        pre_tag_text = tag_text

                
f.close()

## Reformat: remove instances, separate header sections

In [12]:
def reformat(input_path, output_path):
                
    in_f = open(input_path, 'r', encoding='utf8')
    out_f = open(output_path, 'w', encoding='utf8', newline = '')

    csv_r = csv.reader(in_f)
    next(csv_r, None) # skip header
    
    csv_w = csv.writer(out_f)
    header = ['INSTANCE_ID', 'REFERENCE_ID']
    header += ['level_all_num', 'level1_num', 'level2_num', 'level3_num', 'level4_num', 'level5_num', 'level6_num']
    header += ['level1_char', 'level2_char', 'level3_char', 'level4_char', 'level5_char', 'level6_char']
    header += ['ANCHOR_TEXT', 'CONTEXT_PARAGRAPH']
    csv_w.writerow(header)
    
    i = 1
    csv_r = list(csv_r)
    length = len(csv_r)
    for index,row in enumerate(csv_r):
        INSTANCE_ID, REFERENCE_ID, Level1, Level2, Level3, Level4, Level5, Level6, AnchorText, ContextParagraph = row
        if index<length-1:
            _, next_re_id, _, _, _, _, _, _, next_anchor, next_context = csv_r[index+1]
        
        if (REFERENCE_ID == next_re_id) and (next_anchor.startswith(AnchorText))and(ContextParagraph[:20] == next_context[:20]):
            continue     
        
        level1_num = '.'
        level1_txt = ''
        level2_num = '.'
        level2_txt = ''
        level3_num = '.'
        level3_txt = ''
        level4_num = '.'
        level4_txt = ''
        level5_num = '.'
        level5_txt = ''
        level6_num = '.'
        level6_txt = ''
        section_num = '.'
        section_txt = ''
        
        if Level1 != '-':
            if Level1 == 'Preamble' or Level1 == 'Executive Summary':
                level1_num = '0'
                level1_txt = Level1
            else:
                level1_num = Level1.split(' ', 2)[1]
                level1_txt = ' '.join(Level1.split(' ', 2)[:2] + ['|'] + Level1.split(' ', 2)[2:])
            section_num = level1_num
            section_txt = level1_txt
            
        if Level2 != '-':
            level2_num = Level2.split(' ', 1)[0].split('.')[1]
            level2_txt = Level2.split(' ', 1)[1]
            section_num, section_txt = Level2.split(' ', 1)
        if Level3 != '-':
            level3_num = Level3.split(' ', 1)[0].split('.')[2]
            level3_txt = Level3.split(' ', 1)[1]
            section_num, section_txt = Level3.split(' ', 1)
        if Level4 != '-':
            level4_num = Level4.split(' ', 1)[0].split('.')[3]
            level4_txt = Level4.split(' ', 1)[1]
            section_num, section_txt = Level4.split(' ', 1)
        if Level5 != '-':
            level5_num = Level5.split(' ', 1)[0].split('.')[4]
            level5_txt = Level5.split(' ', 1)[1]
            section_num, section_txt = Level5.split(' ', 1)
        if Level6 != '-':
            level6_num = Level6.split(' ', 1)[0].split('.')[5]
            level6_txt = Level6.split(' ', 1)[1]
            section_num, section_txt = Level6.split(' ', 1)
            
        new_row = [str(i), REFERENCE_ID]
        new_row += [section_num, level1_num, level2_num, level3_num, level4_num, level5_num, level6_num]
        new_row += [level1_txt, level2_txt, level3_txt, level4_txt, level5_txt, level6_txt]
        new_row += [AnchorText, ContextParagraph]
        csv_w.writerow(new_row)
        
        i += 1
        
    in_f.close()
    out_f.close()

In [13]:
new_csv_path = 'LEAD_2013_citation_table.csv'
reformat(csv_path, new_csv_path)